![puma logo](https://github.com/nasa/puma/raw/main/doc/source/puma_logo.png)
# PuMA problem set - Problem 1 starter code

The purpose of this tutorial is to introduce the user to the thermal conductivity module of the PuMA software

# Installation setup and imports

If you are running this jupyter notebook locally on your machine, then you don't need to run any setup, granted that you installed PuMA using the installer.sh script. 

If you are running this notebook online on Google Colab and you only need to run the python tutorials (except for the one about Weaves), the following command is enough to setup the environment. 

Note that the first time this runs, it may take a couple of minutes.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install 'git+https://github.com/nasa/puma'
    !pip install -q piglet pyvirtualdisplay
    !apt-get -qq install xvfb

# Importing pumapy


In [ ]:
import numpy as np
import pumapy as puma
import pyvista as pv
import scipy.ndimage as nd
import os
import sys
if 'google.colab' in str(get_ipython()):
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(600, 400))
    display.start()  # necessary for pyvista interactive plots
    
else:  # NORMAL JUPYTER NOTEBOOK
    # for interactive slicer (only static allowed on Colab)
    %matplotlib widget

### Calculation of Thermal Conductivity

This section is designed to produce a simple working example that you can modify for Problem #1 part b

In [ ]:
# Create a workspace from a shape. The workspace is now a 100x10x1 array
workspace = puma.Workspace.from_shape((100,10,10))
workspace.voxel_length = 0.01

# By default, the from_shape function assigns a value of 0 to all cells. 
# We will assign half of the domain to have a value of 1 by modifying the underlying numpy array.
# Note: If you are unfamiliar with how to modify numpy array values, look here: https://numpy.org/devdocs/user/quickstart.html
workspace.matrix[50:,:,:] = 1

# We can see our values by using the plot_slices function
slices = puma.plot_slices(workspace)

# The first step of running a thermal conductivity calculation is assigning conductivity values to each workspace value. 
# We do this using a conductivity map, which maps material ID ranges to thermal conductivity values

# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.IsotropicConductivityMap()
# First, we set the conductivity of the left side to be 1
cond_map.add_material((0, 0), 1)
# Next we set the conductivity of the right side to be 10
cond_map.add_material((1, 1), 10)

# The thermal conductivity calculation can be run for each of the three simulation directions. 
# For each simulation, a temperature gradient is forced in the simulation direction, and converged to steady state

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

k_eff_x, T_x, q_x = puma.compute_thermal_conductivity(workspace, cond_map, 'x', 's', tolerance=1e-4, solver_type='cg')
k_eff_y, T_y, q_y = puma.compute_thermal_conductivity(workspace, cond_map, 'y', 's', tolerance=1e-4, solver_type='cg')
k_eff_z, T_z, q_z = puma.compute_thermal_conductivity(workspace, cond_map, 'z', 's', tolerance=1e-4, solver_type='cg')


print("Effective thermal conductivity tensor:")
print(k_eff_x)
print(k_eff_y)
print(k_eff_z)


### Calculation of Thermal Conductivity on real materials
Below is an example of calculating the thermal conductivity of a real material, to be used in Problem 1 part c. The sample size is much smaller than would be used in a production case, but it serves as a demonstration


In [ ]:
# Generating a small fibrous structure
workspace = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1e-6)

# If you would like to see what this sample looks like, uncomment the line below and run this block. Note that this may
puma.render_contour(workspace, (90,255), notebook=True) # Set notebook=False if running on local machine


# Downscaling the sample a little bit so that it runs faster
workspace.resize((100,100,100), segmented=False)

# We can see our values by using the plot_slices function
# slices = puma.plot_slices(workspace)

# The first step of running a thermal conductivity calculation is assigning conductivity values to each workspace value. 
# We do this using a conductivity map, which maps material ID ranges to thermal conductivity values

# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.IsotropicConductivityMap()
# First, we set the conductivity of the void phase to be 0.0257, or that of air and STP
cond_map.add_material((0, 89), 0.0257)
# Next we set the conductivity of the solid phase to be 12, roughly correct for carbon fibers
cond_map.add_material((90, 255), 12)

# The thermal conductivity calculation can be run for each of the three simulation directions. 
# For each simulation, a temperature gradient is forced in the simulation direction, and converged to steady state

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

k_eff_x, T_x, q_x = puma.compute_thermal_conductivity(workspace, cond_map, 'x', 's', tolerance=1e-4, solver_type='cg')
# k_eff_y, T_y, q_y = puma.compute_thermal_conductivity(workspace, cond_map, 'y', 's', tolerance=1e-4, solver_type='cg')
# k_eff_z, T_z, q_z = puma.compute_thermal_conductivity(workspace, cond_map, 'z', 's', tolerance=1e-4, solver_type='cg')


print("Effective thermal conductivity tensor:")
print(k_eff_x)
# print(k_eff_y)
# print(k_eff_z)